### Tuning a filter / channel 

This notebook shows how to investigate the impact of changing the filter/selection parameters for the Infant SN channel.

Selection has three elements:
- Filter using the XshooterFilter (a close child of DecentFilter).
- Catalog matching to a set of standard catalogs.
- Distance and age calculations using T2InfantCatalogEval. 

The effects are investigated through processing _either_ a set of alerts belonging to known, nearby SNe _or_ alerts obtained through processing all alerts from a certain time-range. Through running the notebook twice it is thus possible to evaluate what fraction of the comparison SNe (and at what age) would be selected, and compare to the number of background alerts selected (likely, but not necessarily, uninteresting).

The three main notebook sections are:
A. Configurations of unit parameters.
B. Retrieval of test alerts through _either_ section B1 _or_ B2.
C. Processing of alerts (which can take a while for background tests).

The notebook assumes that the dev/0.8.2 branch of Ampel-HU-astro has been installed. 

In [ ]:
# Archive access is granted through https://ampel.zeuthen.desy.de/live/dashboard/tokens
token = os.environ["ARCHIVE_TOKEN"]

In [ ]:
import requests, os

In [ ]:
header = {"Authorization": "bearer "+token}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'
from ampel.log.AmpelLogger import AmpelLogger
from ampel.ztf.t0.DecentFilter import DecentFilter
from ampel.contrib.hu.t0.XShooterFilter import XShooterFilter
from ampel.ztf.t2.T2CatalogMatch import T2CatalogMatch
from ampel.contrib.hu.t2.T2InfantCatalogEval import T2InfantCatalogEval
from ampel.view.LightCurve import LightCurve
from ampel.view.T2DocView import T2DocView

from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier
from ampel.ztf.ingest.ZiDataPointShaper import ZiDataPointShaper
from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

In [ ]:
# The logger can be used to display information to stdout, or to file, and at different levels
logger = AmpelLogger.get_logger()

## A. Define filter parameters
Actions are carried out by units (python modules), where the behaviour is typically governed by parameters. You can look at the docstring and code for the used units in the cells below.

In [ ]:
# A filter designed to find transients with a recent non-detection...
XShooterFilter??

In [ ]:
# ...for the most part it, in turn, uses the DecentFilter
DecentFilter??

In [ ]:
# The position of the alert is used to match to a set of standard catalogs
T2CatalogMatch??

In [ ]:
# The combined lightcurve and redshift (through catalog match) info is used to evaluate 
# the transient
T2InfantCatalogEval??

The first set of parameters regulate how alerts are _filtered_. Alerts which are rejected are not considered for further processing.

In [ ]:
# The following settings define the filter behaviour
filter_config = {
    "min_ndet": 1,          # Min number of detections
    "min_tspan": -1.,        # Min total detection age in alert
    "max_tspan": 5.,        # Max total detection age in alert
    "min_archive_tspan": -1,        # Min total detection age in IPAC DB
    "max_archive_tspan": 5.,        # Max total detection age in IPAC DB
    "min_rb": 0.3,             # real bogus score
    "min_drb": 0.995,         # deep learning real bogus score 
    "max_fwhm": 5.5,        # sexctrator FWHM (assume Gaussian) [pix]
    "max_elong": 1.4,       # Axis ratio of image: aimage / bimage
    "max_magdiff": 1,       # Difference: magap - magpsf [mag]
    "max_nbad": 0,          # number of bad pixels in a 5 x 5 pixel stamp
    "min_sso_dist": 20,     # distance to nearest solar system object [arcsec]
    "min_gal_lat": 14,      # minium distance from galactic plane. Set to negative to disable cut.
    # Gaia rejection (based on catalog matching)
    "gaia_rs": 0,            # Disabled for now!
    "gaia_pm_signif": 3,
    "gaia_plx_signif": 3,
    "gaia_veto_gmag_min": 9,
    "gaia_veto_gmag_max": 20,
    "gaia_excessnoise_sig_max": 999,
    # PS1 rejection (based on alert content)
    "ps1_sgveto_rad": 1,
    "ps1_sgveto_th": 0.8,
    "ps1_confusion_rad": 3,
    "ps1_confusion_sg_tol": 0.1,
    # The paraeters below are only available for the Xshooter filter:
    "max_dec": 999,         # Max declination, shortcuts to transients visible from the south
    "det_within": 1.,     # Recent first detection 
    "ul_within": 1.5,         # A previous upper limit with this time (days)
    # Updated parameters based on infant detections spring 2021. Defaults conservative
    "max_chipsf": 4,        # Best guess value 2
    "max_seeratio": 2,      # Best guess value 1.3
    "min_sumrat": 0.6,      # Best guess value 0.8    min_tspan: -666

}

Next, a series of catalogs, together with matching radius, are defined.

In [ ]:
cat_conf = { 'catalogs': 
            {'SDSS_spec': 
             {'use': 'extcats',
              'rs_arcsec': 10,
             'keys_to_append': ['z', 'bptclass', 'subclass']
             },
              'NEDz': {
                  'use': 'catsHTM',
                   'rs_arcsec': 10,
                   'keys_to_append': ['ObjType', 'Velocity', 'z']
              },
              'SDSSDR10': {
                  'use': 'catsHTM',
                   'rs_arcsec': 3,
                   'keys_to_append': ['type', 'flags']
              },
              'brescia': {
                  'use': 'extcats',
                   'rs_arcsec': 3,
                   'keys_to_append': ['subclass', 'z']
              },
              'milliquas': {
                  'use': 'extcats',
                   'rs_arcsec': 3,
                   'keys_to_append': ['broad_type', 'name', 'redshift', 'qso_prob']
              },
              'GAIADR2': {
                  'use': 'catsHTM',
                   'rs_arcsec': 30,
                   'keys_to_append': ['Mag_G', 'PMRA', 'ErrPMRA', 'PMDec', 'ErrPMDec',
                                    'Plx', 'ErrPlx', 'ExcessNoise', 'ExcessNoiseSig']
              },
              'CRTS_DR1': {
                  'use': 'extcats',
                   'rs_arcsec': 3,
                   'keys_to_append': ['VarType']
              },
              'AAVSOVSX': {
                  'use': 'extcats', 
                  'rs_arcsec': 3, 
                  'keys_to_append': ['TYPE']},
              'LAMOSTDR4': {
                  'use': 'extcats',
                   'rs_arcsec': 3,
                   'keys_to_append': ['objtype', 'class', 'subclass', 'snrg']
              },
              'GLADEv23': {
                  'use': 'extcats',
                   'rs_arcsec': 10,
                   'keys_to_append': ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3']
              },
              'allWISE_skymotion': {
                  'use': 'extcats',
                   'rs_arcsec': 30,
                   'keys_to_append': ['pmRA', 'e_pmRA', 'pmDE', 'e_pmDE']
              },
              'NEDz_extcats': {
                  'use': 'extcats',
                   'rs_arcsec': 60,
                   'post_filter': {'z': {'$lte': 0.03, '$gte': 0.002}},
                   'keys_to_append': ['ObjType', 'Velocity', 'z']
              }
           },
    "resource": {
        'ampel-ztf/catalogmatch': 'https://ampel.zeuthen.desy.de/api/catalogmatch/'
    }
}

The evaluation unit is defined as follows.

In [ ]:
eval_config = {
    'max_age': 3,
    'det_filterids': [1,2],
    'max_kpc_dist': 60,
    'min_absmag': -17,
    'max_redshift': 0.03,
    't2_dependency': [],
}

We will now create the unit instances (which will be used below).

In [ ]:
# Create the filter instance
t0filter = XShooterFilter( **filter_config, logger=logger )
t0filter.post_init()
t2catalogs = T2CatalogMatch(**cat_conf, logger=logger)
t2eval = T2InfantCatalogEval(**eval_config, logger=logger)

### B. Selecting which alerts to run against

Alerts will be assumed through a _resume token_, which points to a specific set of alerts. In sec B1 we will obtain this for a static list of known nearby SNe, in B2 for a list of alerts found in a given time-range.>

### B1. Using a topic token
These are permanent pointers to list of alerts. Here we will first create a stream from this list. 



topic: Y9R82mkcL1SJq16rTt8_vh-_dx0V7rwEHbGgSgUMlqI

description: A list of the final alert of 27 SNe which were detected early during 2020. Created May 24 2022.

topic: fz6nmCJwKSdZXhzm3JuBTlMY3xjPsdhkobVLvZzzmt8

description: A list of 2565 alerts from 27 2020 SNe which were detected early. Created May 24 2022.


In [ ]:
topic_token = "fz6nmCJwKSdZXhzm3JuBTlMY3xjPsdhkobVLvZzzmt8"

In [ ]:
body = {
    "topic": topic_token, 
    "chunk_size": 100
}

In [ ]:
response = requests.post(f"{base_url}/streams/from_topic", headers={"Authorization": "bearer "+token}, json=body )

In [ ]:
if not response.ok:
    print('Accessing stream from topic not successful')
    print(response)

In [ ]:
resume_token = response.json()['resume_token']

In [ ]:
# Alerts are now being queued in the archive, and can be retrieved through this token
resume_token

### B2. Quering random alerts
We will here grab a random set of alerts to check throughput. Note that we already in the first stage restrict the query based on drb, magpsf and number of detections.

In [ ]:
query = {
  "jd": {
    "$gt": 2459500.5,
    "$lt": 2459511.5
  },
  "candidate": {
    "drb": {
      "$gt": 0.995
    },
    "magpsf": {
      "$gt": 16
    },
    "ndethist": {
      "$gt": 0,
      "$lte": 2
    },
  }
}

In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'
header = {"Authorization": "bearer "+os.environ["ARCHIVE_TOKEN"]}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
response.ok

In [ ]:
resume_token = response.json()['resume_token']

It will take some time to line up a large query. 

## C. Iterate through filters from the stream, checking whether the filter would accept them

In [ ]:
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream": resume_token}   # From above

In [ ]:
# AMPEL internal - regulates how to parse the ZTF alert schema
alertloader = ZTFArchiveAlertLoader(**config)
shaper = ZiDataPointShaper(logger=logger)

In [ ]:
# Store some evaluation results
process_results = {}

The next step will iterate through all alerts.

In [ ]:
for alert in alertloader.get_alerts():
    if not alert['objectId'] in process_results.keys():
        process_results[alert['objectId']] = []
        
    # Check wether the alert passes the initial filter. If not we go on to the next
    filter_accept = t0filter.process( ZiAlertSupplier.shape_alert_dict( alert, [] ) )
    if not filter_accept:
        process_results[alert['objectId']].append( {'candid':alert['candid'], 
                                        'jd':alert['candidate']['jd'], 'eval':0} )
        continue

    # AMPEL internal - create a LightCurve object from alert
    ampel_alert = ZiAlertSupplier.shape_alert_dict(alert)
    dps = shaper.process(ampel_alert.datapoints, stock=alert['objectId'])
    lc = LightCurve.build({'stock':'foo', "link":14}, dps)
    
    # Match the object to the Zeuthen catalog archive
    catout = t2catalogs.process(lc.get_photopoints()[0])
    
    # Format the catalog output, and provide this and the lc to the evaluation unit.
    catview = T2DocView(stock='foo', unit='T2CatalogMatch', link=14, tag=[], code=0, t2_type=0, meta=[{'tier':2,'code':0}], body=[catout])
    evalout = t2eval.process(lc, [catview])

    # Check result and divide according to
    # 1. Passed filter criteria, but not T2InfantEval criteria.
    # 2. Passes also T2InfantEval criteria
    result = {}
    first_det = min(lc.get_values('jd'))
    result = {'candid':alert['candid'],'jd':alert['candidate']['jd'], 'jd_det':min(lc.get_values('jd'))}
    if evalout['action']:
        result['eval'] = 2
    else:
        result['eval'] = 1
    process_results[alert['objectId']].append( result )
        


In [ ]:
print('Number of unique transients selected:', len( set( [s[1] for s in summary if s[-1]] ) ))

In [ ]:
print('Total number of alertsselected:', len( ( [s[1] for s in summary if s[-1]] ) ) )

In [ ]:
print('Alerts examined', len(summary))

Example based on the filter settings last used above:
- Processing 50 days of alerts: 1096089 alerts pass query candidates, of these yielded 8179 transient selections from 8215 selected alerts (~160/day).
- Processing the 2565 alerts of the 27 2020 infant SNe, 38 alerts of 15 SNe were selected. 

In [ ]:
alert = next(alertloader.get_alerts())

In [ ]:
alert

In [ ]:
ampel_alert = ZiAlertSupplier.shape_alert_dict(alert)

In [ ]:
dps = shaper.process(ampel_alert.datapoints, stock=alert['objectId'])

In [ ]:
t2catalogs.__

In [ ]:
catout = t2catalogs.process(lc.get_photopoints()[0] )

In [ ]:
t2eval.process??

In [ ]:
lc = LightCurve({'stock':'foo'}, dps)

In [ ]:
t2eval.process(lc, [catview])

In [ ]:
catview = T2DocView(stock='foo', unit='T2CatalogMatch', link=14, tag=[],code=0, t2_type=0, meta=[{'tier':2,'code':0}], body=[catout])

In [ ]:
catview??

In [ ]:
catview.body

In [ ]:
catview.get_payload()

In [ ]:
catout

In [ ]:
dps

In [ ]:
lc = LightCurve.build({'stock':'foo', "link":14}, dps)

In [ ]:
min(lc.get_values('jd'))

In [ ]:
lc??

In [ ]:
counter = [0,0,0,0]

In [ ]:
for sn, results in process_results.items():
    counter[0] += 1
    counter[1] += len(results)
    passed = [a['jd']-a['jd_det'] for a in results if a['eval']>0]
    print(sn, passed, len(results))
    if len(passed)>0:
        print('... detected at phase', min(passed))
        counter[2]+=1
    calc = [a['jd']-a['jd_det'] for a in results if a['eval']>1]
    if len(calc)>0:
        print('... eval passed at phase', min(calc))
        counter[3]+=1
    

In [ ]:
print('Out of {} targets, which distributed {} alerts, {} were accepted by the filter and {} also by the Infant evaluation unit.'.format(*counter))